# 📝 Fine-Tuning T5-Small on Dolly-15K for Summarization

This notebook demonstrates how to fine-tune the `google-t5/t5-small` model on the [`databricks/databricks-dolly-15k`](https://huggingface.co/datasets/databricks/databricks-dolly-15k) dataset for **abstractive text summarization** using Hugging Face’s `Transformers`, `Datasets`, and `Accelerate` libraries.

## 🔧 Project Details

- **Model**: `google-t5/t5-small` (60M parameters)
- **Dataset**: `databricks/databricks-dolly-15k`
  - **Input**: `context`
  - **Target**: `response`
- **Task**: Abstractive Summarization
- **Libraries Used**: 🤗 Transformers, Datasets, Accelerate, PyTorch, NLTK
- **Training Method**: Custom training loop using `Accelerate`
- **Evaluation Metric**: ROUGE (ROUGE-1, ROUGE-2, ROUGE-L, ROUGE-Lsum)

## 📊 Training Results (ROUGE Scores)

| Epoch | ROUGE-1 | ROUGE-2 | ROUGE-L | ROUGE-Lsum |
|-------|---------|---------|---------|------------|
| 0     | 31.16   | 18.47   | 28.42   | 28.90      |
| 1     | 31.39   | 18.63   | 28.65   | 29.16      |
| 2     | 31.41   | 18.61   | 28.72   | 29.19      |
| 3     | 31.46   | 18.65   | 28.76   | 29.25      |
| 4     | 31.46   | 18.65   | 28.76   | 29.25      |

> The model showed consistent improvements over the first few epochs, stabilizing after epoch 3.

## 🚀 How to Use

You can load the fine-tuned model locally for summarization:

```python
from transformers import pipeline

summarizer = pipeline("summarization", model="results-t5-finetuned-squad-accelerate", tokenizer="results-t5-finetuned-squad-accelerate")

text = "Coffee drinks are made by brewing water with ground coffee beans..."
summary = summarizer(text, max_length=50)[0]["summary_text"]
print(summary)


In [ ]:
!pip install --upgrade datasets

### Dataset loading and split for train, test, validation

In [2]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("databricks/databricks-dolly-15k")

#  Split into train + temp (for validation and test)
train_test_split = ds["train"].train_test_split(test_size=0.2, seed=42)
train_ds = train_test_split["train"]
temp_ds = train_test_split["test"]

#  Split temp into validation + test
val_test_split = temp_ds.train_test_split(test_size=0.5, seed=42)
val_ds = val_test_split["train"]
test_ds = val_test_split["test"]


ds_split = {
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
}


README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [3]:
# Storing split value on dataset

from datasets import DatasetDict

ds = DatasetDict(ds_split)
ds


DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 12008
    })
    validation: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 1501
    })
    test: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 1502
    })
})

In [4]:
# Remove empty rows
def has_non_empty_context(example):
    return example["context"].strip() != ""

# Apply the filter to each split and rebuild the DatasetDict
ds = {
    split: dataset.filter(has_non_empty_context)
    for split, dataset in ds.items()
}



Filter:   0%|          | 0/12008 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1501 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1502 [00:00<?, ? examples/s]

In [5]:
# visualize first 3 rows
for example in ds["train"].select(range(3)):
    print("Context:\n", example["context"])
    print("Response:\n", example["response"])
    print("=" * 80)


Context:
 Dharma Sabha was formed in 1830 in Calcutta by Radhakanta Deb. The organization was established mainly to counter the ongoing social reform movements led by protagonists such as Raja Ram Mohun Roy and Henry Derozio. More specifically, the impetus of forming the organization came from a new law enacted by the colonial British rule which banned the barbaric practice of burning widows alive (sati) in the country; the focus of the new association was to repel the law which was seen as an intrusion by the British into the religious affairs of the indigenous people by some sections of the Hindu community. The Dharma Sabha filed an appeal in the Privy Council against the ban on Sati by Lord William Bentinck as, according to them, it went against the assurance given by George III of non-interference in Hindu religious affairs; however, their appeal was rejected and the ban on Sati was upheld in 1832. It published a newspaper called Samachar Chandrika.

The Dharma Sabha campaigned aga

In [8]:
ds

{'train': Dataset({
     features: ['instruction', 'context', 'response', 'category'],
     num_rows: 3573
 }),
 'validation': Dataset({
     features: ['instruction', 'context', 'response', 'category'],
     num_rows: 453
 }),
 'test': Dataset({
     features: ['instruction', 'context', 'response', 'category'],
     num_rows: 441
 })}

In [6]:
from datasets import DatasetDict

# Convert dict back to DatasetDict
ds = DatasetDict(ds)

# Now set format and convert to pandas
ds.set_format("pandas")
ds_df = ds["train"][:]

# Show counts for top 20 categories
ds_df["category"].value_counts()[:20]


category
closed_qa                 1424
information_extraction    1203
summarization              946
Name: count, dtype: int64

In [11]:
# # IF there are different type of language then we can concat them and feed the model
# # for example combining the English and Spanish reviews as a single DatasetDict object. 🤗 Datasets provides a handy concatenate_datasets() function that (as the name suggests) will stack two Dataset objects on top of each other.
# from datasets import concatenate_datasets, DatasetDict

# books_dataset = DatasetDict()

# for split in english_books.keys():
#     books_dataset[split] = concatenate_datasets(
#         [english_books[split], spanish_books[split]]
#     )
#     books_dataset[split] = books_dataset[split].shuffle(seed=42)

# # Peek at a few examples
# show_samples(books_dataset)

There are many model for text summarization, if we have different type of language then we need to use certain type of models which support Multilingual.my dataset is monolingual so i can use t5, bart and other models. But if our dataset is Multilingual, then we need to use mT5, mBART-50 models, which support Multilingual

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

2025-06-06 17:17:52.014597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749230272.200102      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749230272.253223      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

#### Preprocess the dataset

In [8]:
def preprocess_function(examples):
    contexts = [c if isinstance(c, str) else "" for c in examples["context"]]
    responses = [r if isinstance(r, str) else "" for r in examples["response"]]

    model_inputs = tokenizer(
        contexts,
        max_length=512,
        truncation=True,
    )
    labels = tokenizer(
        responses,
        max_length=30,
        truncation=True,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [9]:
tokenized_datasets = ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/3573 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

In [10]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3573
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 453
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 441
    })
})

### 📏 ROUGE Score for Summarization Evaluation

ROUGE is a metric that evaluates how much a generated summary overlaps with a reference summary using precision, recall, and F1-score.  
- **Recall**: Measures how much of the reference summary is covered by the generated one.  
- **Precision**: Measures how much of the generated summary is relevant to the reference.  
- **F1**: Harmonic mean of precision and recall.

Example:  
Generated → "I absolutely loved reading the Hunger Games"  
Reference → "I loved reading the Hunger Games"  
→ 6 overlapping words → Recall = 1.0, Precision = 0.6, F1 ≈ 0.75

Install with: `!pip install rouge_score`


In [12]:
# !pip install rouge_score evaluate

In [13]:
import evaluate

rouge_score = evaluate.load("rouge")

In [14]:
#Check

generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

### 🧪 Lead-3 Baseline for Summarization

A simple baseline for summarization is the **lead-3** method: return the first 3 sentences of the article.  
To handle sentence boundaries accurately (e.g. “U.S.” vs full stop), we use the `nltk` library:

```bash
!pip install nltk


In [16]:
# !pip install nltk

In [17]:
# download the punctuation rules:
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [19]:
from nltk.tokenize import sent_tokenize

def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


In [20]:
# Convert to pandas first if needed
ds.set_format("pandas")
df = ds["train"][:]

# Use .loc or .iloc to access individual string values
print(three_sentence_summary(df.loc[1, "context"]))


The Transportation Security Administration (TSA) is an agency of the United States Department of Homeland Security (DHS) that has authority over the security of transportation systems within, and connecting to the United States.
It was created as a response to the September 11 attacks to improve airport security procedures and consolidate air travel security under a dedicated federal administrative law enforcement agency.


In [21]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["context"]]
    return metric.compute(predictions=summaries, references=dataset["response"])


***Calculate ROUGE Scores*** <br>
We evaluate the summaries on a validation set and get ROUGE-1, ROUGE-2, ROUGE-L, and ROUGE-Lsum metrics: <br>

***rouge1***: Matches at the word level.<br>

***rouge2***: Matches at the bigram level (two-word phrases) — this is usually lower because exact bigram overlap is rare.<br>

***rougeL***: Measures longest common subsequences — shows fluency and sentence structure match.<br>

***rougeLsum***: Variant better suited for summaries with multiple sentences.<br>

In [22]:
import pandas as pd

score = evaluate_baseline(ds["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = {rn: round(score[rn] * 100, 2) for rn in rouge_names}
print(rouge_dict)


{'rouge1': 31.05, 'rouge2': 19.1, 'rougeL': 26.11, 'rougeLsum': 27.62}


# Fine-tuning T5 with Accelerate

##### Preparing for the training

In [23]:
tokenized_datasets.set_format("torch")

In [24]:
# Transformers provides a DataCollatorForSeq2Seq collator that will dynamically pad the inputs and the labels for us.
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size
)

In [26]:
# setting optimizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [27]:
# Setting Accelerator
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [28]:
# Setting learning rate, it will adjust while training
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

For post-processing, we need a function that splits the generated summaries into sentences that are separated by newlines. This is the format the ROUGE metric expects, and we can achieve this with the following snippet of code:

In [29]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [30]:
output_dir = "results-t5-finetuned-squad-accelerate"

In [32]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        # repo.push_to_hub(
        #     commit_message=f"Training in progress epoch {epoch}", blocking=False
        # )

  0%|          | 0/2235 [00:00<?, ?it/s]

Epoch 0: {'rouge1': 31.1618, 'rouge2': 18.4746, 'rougeL': 28.4243, 'rougeLsum': 28.8968}
Epoch 1: {'rouge1': 31.3859, 'rouge2': 18.6263, 'rougeL': 28.6527, 'rougeLsum': 29.1561}
Epoch 2: {'rouge1': 31.4089, 'rouge2': 18.6144, 'rougeL': 28.7171, 'rougeLsum': 29.1906}
Epoch 3: {'rouge1': 31.4601, 'rouge2': 18.6477, 'rougeL': 28.7622, 'rougeLsum': 29.2484}
Epoch 4: {'rouge1': 31.4601, 'rouge2': 18.6477, 'rougeL': 28.7622, 'rougeLsum': 29.2484}


## Checking the model performance

In [36]:
from transformers import pipeline


model_dir = "results-t5-finetuned-squad-accelerate"

# Load summarization pipeline with your model
summarizer = pipeline("summarization", model=model_dir)



# Example function to summarize a test context
def print_summary(idx):
    review = ds["test"][idx]["context"]  # This should now be a string
    true_summary = ds["test"][idx]["response"]

    summary = summarizer(
        review, max_length=50, clean_up_tokenization_spaces=True
    )[0]["summary_text"]

    print(f">>> Context:\n{review}")
    print(f"\n>>> Ground Truth Summary:\n{true_summary}")
    print(f"\n>>> Model Summary:\n{summary}")


# Try on one test example
ds.reset_format()
print_summary(0)


Device set to use cuda:0


>>> Context:
Coffee drinks are made by brewing water with ground coffee beans. The brewing is either done slowly by drip, filter, French press, cafetière or percolator, or done very quickly under pressure by an espresso machine. When put under the pressure of an espresso machine, the coffee is termed espresso while slow-brewed coffees are generally termed brewed coffee. While all coffee drinks are based on either coffee or espresso, some drinks add milk or cream, some are made with steamed milk or non-dairy milks, or add water (like the americano). Upon milk additions, coffee's flavor can vary with different syrups or sweeteners, alcoholic liqueurs, and even combinations of coffee with espresso or tea. There are many variations to the basic coffee or espresso bases.

>>> Ground Truth Summary:
Different methods of brewing coffee:
- Slow-brewed coffee
  - Drip
  - Filter
  - French press
  - Cafetière
  - Percolator
- Espresso machine

>>> Model Summary:
coffee drinks are made by brewing